In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu

# Define an encoder using Convolutional layers
def encoder(input_shape):
    input_layer = Input(shape=input_shape)
    conv1 = Conv2D(64, (3, 3), activation=relu, padding='same')(input_layer)
    conv2 = Conv2D(128, (3, 3), activation=relu, padding='same')(conv1)
    encoded = GlobalAveragePooling2D()(conv2)
    return Model(inputs=input_layer, outputs=encoded, name='encoder')

# Define a transformer model
def transformer(input_shape):
    input_layer = Input(shape=input_shape)
    # Your transformer layers here
    # Attention mechanisms, multi-head attention, positional encodings, etc.
    transformed = # Transformer layers
    return Model(inputs=input_layer, outputs=transformed, name='transformer')

# Define a decoder using Dense layers
def decoder(latent_dim, output_dim):
    input_layer = Input(shape=(latent_dim,))
    dense1 = Dense(128, activation=relu)(input_layer)
    dense2 = Dense(256, activation=relu)(dense1)
    output_layer = Dense(output_dim, activation='softmax')(dense2)
    return Model(inputs=input_layer, outputs=output_layer, name='decoder')

# Create an encoder-decoder-transformer model
input_shape = (256, 256, 3)
latent_dim = 512
output_dim = 10  # Example for a classification task
input_data = Input(shape=input_shape)
encoded = encoder(input_shape)(input_data)
transformed = transformer((latent_dim,))(encoded)
decoded = decoder(latent_dim, output_dim)(transformed)

model = Model(inputs=input_data, outputs=decoded, name='encoder_decoder_transformer')

# Compile and train the model
# You'll need to define your loss function, optimizer, and dataset for training

# Example training loop:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(train_dataset, epochs=10, validation_data=val_dataset)


In [ ]:
# Cyclic Learning Rate Schedule class CyclicLR(Callback): def __init__(self, base_lr=1e-4, max_lr=3e-4, step_size=2000., mode='triangular2'): super(CyclicLR, self).__init__() self.base_lr = base_lr self.max_lr = max_lr self.step_size = step_size self.mode = mode self.clr_iterations = 0. self.trn_iterations = 0. if self.mode not in ['triangular', 'triangular2', 'exp_range']: raise KeyError("mode must be one of 'triangular', 'triangular2', or 'exp_range'")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2DTranspose, Reshape, Dense
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import os
import urllib.request
import tarfile
import shutil
from vit_keras import vit
from tensorflow.keras.callbacks import LearningRateScheduler, Callback, EarlyStopping
from tensorflow.keras.regularizers import l2

# Download STL-10 dataset (if not already downloaded)
url = 'http://ai.stanford.edu/~acoates/stl10/stl10_binary.tar.gz'
file_name = 'stl10_binary.tar.gz'

if not os.path.exists(file_name):
    urllib.request.urlretrieve(url, file_name)

# Extract the dataset (if not already extracted)
data_dir = 'stl10_binary'
file_names = ['train_X.bin', 'train_y.bin', 'test_X.bin', 'test_y.bin']

if os.path.exists(data_dir):
    shutil.rmtree(data_dir)  # Delete the existing folder if it exists

tar = tarfile.open(file_name, "r:gz")
tar.extractall()
tar.close()

# Load the dataset
x_train_path = os.path.join(data_dir, file_names[0])
y_train_path = os.path.join(data_dir, file_names[1])
x_test_path = os.path.join(data_dir, file_names[2])
y_test_path = os.path.join(data_dir, file_names[3])

x_train = np.fromfile(x_train_path, dtype=np.uint8).reshape(-1, 3, 96, 96).transpose(0, 2, 3, 1)
y_train = np.fromfile(y_train_path, dtype=np.uint8) - 1
x_test = np.fromfile(x_test_path, dtype=np.uint8).reshape(-1, 3, 96, 96).transpose(0, 2, 3, 1)
y_test = np.fromfile(y_test_path, dtype=np.uint8) - 1

# Preprocessing: normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Define the VIT Autoencoder architecture
latent_dim = 128

# Encoder (Vision Transformer)
inputs = Input(shape=(96, 96, 3))
x = vit.vit_l32(image_size=96, activation='gelu', pretrained=False, include_top=False, pretrained_top=False)(inputs)
x = Reshape((-1, x.shape[-1]))(x)  # Flatten the sequence of patches
x = tf.keras.layers.GlobalAveragePooling1D()(x)  # Reduce sequence to a single vector
latent_space = Dense(latent_dim, activation='relu', kernel_regularizer=l2(1e-5), kernel_initializer='glorot_uniform')(x)  # Dense layer with L2 regularization

# Decoder
decoder_inputs = Input(shape=(latent_dim,))
x = Dense(6 * 6 * 32, activation='relu', kernel_regularizer=l2(1e-5), kernel_initializer='glorot_uniform')(decoder_inputs)  # Dense layer with L2 regularization
x = Reshape((6, 6, 32))(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', strides=(2, 2), padding='same', kernel_regularizer=l2(1e-5), kernel_initializer='glorot_uniform')(x)
x = Conv2DTranspose(16, (3, 3), activation='relu', strides=(2, 2), padding='same', kernel_regularizer=l2(1e-5), kernel_initializer='glorot_uniform')(x)
outputs = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same', kernel_regularizer=l2(1e-5), kernel_initializer='glorot_uniform')(x)

# AE model
encoder = Model(inputs, latent_space)
decoder = Model(decoder_inputs, outputs)

# Create the autoencoder by connecting the encoder and decoder
autoencoder_output = decoder(encoder(inputs))
autoencoder = Model(inputs, autoencoder_output)

# Reshape inputs and outputs
inputs_reshaped = tf.image.resize(inputs, (24, 24))
autoencoder_output_reshaped = tf.image.resize(autoencoder_output, (24, 24))

# Define the loss function (Autoencoder loss)
reconstruction_loss = tf.keras.losses.mean_squared_error(tf.reshape(inputs_reshaped, (-1, 24 * 24 * 3)),
                                                         tf.reshape(autoencoder_output_reshaped, (-1, 24 * 24 * 3)))

autoencoder_loss = tf.reduce_mean(reconstruction_loss)

# Compile the model
autoencoder.add_loss(autoencoder_loss)
autoencoder.compile(optimizer='adam')

# Cyclic Learning Rate Schedule
class CyclicLR(Callback):
    def __init__(self, base_lr=1e-4, max_lr=3e-4, step_size=2000., mode='triangular2'):
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        if self.mode not in ['triangular', 'triangular2', 'exp_range']:
            raise KeyError("mode must be one of 'triangular', 'triangular2', or 'exp_range'")

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.mode == 'triangular':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x))
        elif self.mode == 'triangular2':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) / float(2  (cycle - 1))
        elif self.mode == 'exp_range':
            gamma = 1 / float(2  (cycle - 1))
            return self.base_lr * (gamma  self.clr_iterations)

    def on_train_begin(self, logs=None):
        logs = logs or {}
        tf.keras.backend.set_value(self.model.optimizer.lr, self.base_lr)

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        tf.keras.backend.set_value(self.model.optimizer.lr, self.clr())

# Set up cyclic learning rate
base_lr = 1e-5
max_lr = 3e-4
batch_size = 128
step_size = 4 * (len(x_train) // batch_size)
clr = CyclicLR(base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='triangular2')

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
epochs = 100
batch_size = 128
history = autoencoder.fit(x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, None), callbacks=[clr, early_stopping])

# Plot the learning curves for loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid()
plt.show()

# Generate and plot some reconstructed images using the VIT Autoencoder
num_samples = 5
random_indices = np.random.randint(0, len(x_test), num_samples)
sample_images = x_test[random_indices]
reconstructed_images = autoencoder.predict(sample_images)

plt.figure(figsize=(10, 4))

for i in range(num_samples):
    plt.subplot(2, num_samples, i + 1)
    plt.imshow(sample_images[i])
    plt.title("Original")
    plt.axis('off')

    plt.subplot(2, num_samples, num_samples + i + 1)
    plt.imshow(reconstructed_images[i])
    plt.title("Reconstructed")
    plt.axis('off')

plt.show()


In [ ]:
import torch
import torch.nn as nn
import timm  # Library for vision models, including SWIN

# Create a simplified SWIN Transformer model
class SwinTransformer(nn.Module):
    def __init__(self, num_classes, img_size, patch_size, embed_dim, num_layers):
        super(SwinTransformer, self).__init__()
        self.patch_embed = timm.models.vision_transformer.patch_embed.PatchEmbed(
            img_size=img_size, patch_size=patch_size, in_chans=3, embed_dim=embed_dim)

        self.transformer = timm.models.vision_transformer.SwinTransformer(
            img_size=img_size, embed_dim=embed_dim, depths=[num_layers] * 4, num_heads=embed_dim // 32,
            num_classes=num_classes, qkv_bias=True)

    def forward(self, x):
        x = self.patch_embed(x)
        x = self.transformer(x)
        return x

# Hyperparameters
num_classes = 1000
img_size = 224
patch_size = 4
embed_dim = 96
num_layers = 12  # Number of layers in each stage of the SWIN Transformer

# Create the SWIN Transformer model
model = SwinTransformer(num_classes, img_size, patch_size, embed_dim, num_layers)

# Input image tensor
input_image = torch.randn(1, 3, img_size, img_size)  # Batch size 1, 3 channels (RGB), img_size x img_size

# Forward pass through the model
output = model(input_image)
print("Output shape:", output.shape)  # Print the shape of the output tensor


In [ ]:
# Develop VAE and AE  with SWIN Transformers

# Import the required libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
import os
import urllib.request
from tensorflow.keras.regularizers import l2
# Load the Swin Transformer model from Keras examples
from keras.applications import SwinTransformer
import urllib.request
import tarfile
from tensorflow.keras.callbacks import EarlyStopping

# Download STL-10 dataset
url = 'http://ai.stanford.edu/~acoates/stl10/stl10_binary.tar.gz'
file_name = 'stl10_binary.tar.gz'

if not os.path.exists(file_name):
    urllib.request.urlretrieve(url, file_name)

# Extract the dataset
tar = tarfile.open(file_name, "r:gz")
tar.extractall()
tar.close()

# Extract the dataset (if not already extracted)
data_dir = 'stl10_binary'
file_names = ['train_X.bin', 'train_y.bin', 'test_X.bin', 'test_y.bin']

# Load the dataset
x_train_path = os.path.join(data_dir, file_names[0])
y_train_path = os.path.join(data_dir, file_names[1])
x_test_path = os.path.join(data_dir, file_names[2])
y_test_path = os.path.join(data_dir, file_names[3])

x_train = np.fromfile(x_train_path, dtype=np.uint8).reshape(-1, 3, 96, 96).transpose(0, 2, 3, 1)
y_train = np.fromfile(y_train_path, dtype=np.uint8) - 1
x_test = np.fromfile(x_test_path, dtype=np.uint8).reshape(-1, 3, 96, 96).transpose(0, 2, 3, 1)
y_test = np.fromfile(y_test_path, dtype=np.uint8) - 1

# Initialize Swin Transformer model
swin_model = SwinTransformer(
    input_shape=(96, 96, 3),
    num_classes=10,  # The number of classes doesn't matter for VAE and AE
    num_heads=4,     # You can adjust the number of heads based on your requirement
    num_transformer_blocks=2,  # You can adjust the number of blocks based on your requirement
)

# Remove the classification head from the model
swin_model = tf.keras.Model(inputs=swin_model.inputs, outputs=swin_model.layers[-2].output)

# Define the VAE architecture based on the Swin Transformer
inputs = Input(shape=(96, 96, 3))
x = swin_model(inputs)

# Latent space
latent_dim = 256
z_mean = Dense(latent_dim, kernel_regularizer=l2(0.01))(x)  # Adding L2 regularization
z_log_var = Dense(latent_dim, kernel_regularizer=l2(0.01))(x)  # Adding L2 regularization

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = tf.keras.layers.Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_inputs = Input(shape=(latent_dim,))
x = Dense(6 * 6 * 64, activation='relu')(decoder_inputs)
x = Reshape((6, 6, 64))(x)
x = Conv2DTranspose(128, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
outputs = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

# VAE model
vae = Model(inputs, outputs)

# Define the loss function for VAE
def vae_loss(inputs, outputs):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(
        tf.reshape(inputs, (-1, 24 * 24 * 3)), tf.reshape(outputs, (-1, 24 * 24 * 3))
    )
    reconstruction_loss *= 24 * 24 * 3
    kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
    kl_loss = tf.reduce_mean(kl_loss, axis=-1)
    kl_loss *= -0.5
    return tf.reduce_mean(reconstruction_loss + kl_loss)

# Compile the VAE model
vae.compile(optimizer=Adam(learning_rate=1e-4), loss=vae_loss)

# AE model (without the latent space)
encoder = Model(inputs, z_mean)
decoder = Model(decoder_inputs, outputs)

# Compile the AE model
decoder.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy')


# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training the VAE model
epochs = 100
batch_size = 128
history_vae = vae.fit(x_train, x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, x_test), callbacks=[early_stopping])

# # Training the AE model
# history_ae = decoder.fit(x_train, x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, x_test), callbacks=[early_stopping])

# # Save the models (optional)
# vae.save("vae_model.h5")
# decoder.save("ae_model.h5")

# # Load the models (optional)
# loaded_vae = tf.keras.models.load_model("vae_model.h5")
# loaded_decoder = tf.keras.models.load_model("ae_model.h5")
import matplotlib.pyplot as plt

# Plot the learning curves for loss
plt.figure(figsize=(10, 6))
plt.plot(history_vae.history_vae['loss'], label='Train Loss')
plt.plot(history_vae.history_vae['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid()
plt.show()
